In [43]:
import pandas as pd

# Analiza receptov v računalniški igri Minecraft

### Uvod in razlage pojmov

Ta projekt iz glavne wiki spletne strani za računalniško igro Minecraft postrga vse recepte za izdelavo predmetov. Te nato shrani v CSV tabelo in jih analizira, a o tem kasneje.

Najprej se dogovorimo za nekaj izrazov:
- Material - predmet, ki je uporabljen v izdelavi drugega predmeta
- Predmet - predmet, ki ga poskušamo sestaviti iz materialov
- Recept - razporeditev do devetih matrialov v kvadratu velikosti 3x3, iz česar nato lahko dobimo nov predmet. Nam ni pomembna sama razporeditev, le koliko je potrebnega vsakega materiala

Prvi korak je naložiti HTML spletne strani na računalnik in ga predelati v datoteko CSV. To naredimo tako, da zaženemo program `main.py`, a sem na repozitorij objavil vse vmesne datoteke, saj je del programa, ki naloži HTML nekoliko nezanesljiv (zelo odvisen od internetne povezave, verzije iskalnika Chrome ipd.). Če vam pri zagonu `main.py` selenium vrača napako, nastavite spremenljivko na začetku programa na `download_from_web = False`.

In [44]:
csv_data = pd.read_csv("data\data.csv", header=0).sort_values("Name").sort_values("Section")
csv_data

,Section,Name,Ingredients,Amounts,Yield
354,Brewing,Fermented_Spider_Eye,Brown_Mushroom;Spider_Eye;Sugar,1;1;1,1
355,Brewing,Glass_Bottle,Glass,3,3
356,Brewing,Glistering_Melon_Slice,Gold_Nugget;Melon_Slice,8;1,1
353,Brewing,Cauldron,Iron_Ingot,7,1
352,Brewing,Brewing_Stand,Blaze_Rod;Cobblestone,1;2,1
...,...,...,...,...,...
318,Utilities,Compass,Iron_Ingot;Redstone_Dust,4;1,1
321,Utilities,Spyglass,Amethyst_Shard;Copper_Ingot,1;2,1
319,Utilities,Lead,Slimeball;String,1;4,2
320,Utilities,Recovery_Compass,Compass;Echo_Shard,1;8,1


Poglejmo si tudi, kateri recepti proizvedejo največ predmetov oz. imajo največji donos:

In [45]:
csv_data.sort_values("Yield", ascending=False)

,Section,Name,Ingredients,Amounts,Yield
202,Decoration_blocks,Iron_Bars,Iron_Ingot,6,16
237,Decoration_blocks,Stained_Glass_Pane,White_Stained_Glass,6,16
287,Transportation,Rail,Iron_Ingot;Stick,6;1,16
195,Decoration_blocks,Glass_Pane,Glass,6,16
402,Materials,Raw_Gold,Block_of_Raw_Gold,1,9
...,...,...,...,...,...
178,Decoration_blocks,Dyed_Candle,Candle;White_Dye,1;1,1
173,Decoration_blocks,Blast_Furnace,Furnace;Iron_Ingot;Smooth_Stone,1;5;3,1
191,Decoration_blocks,Ender_Chest,Eye_of_Ender;Obsidian,1;8,1
187,Decoration_blocks,Enchanting_Table,Book;Diamond;Obsidian,1;2;4,1


Vidimo, da imajo železne rešetke, tračnice in vitraži ter prozorne steklene plošče vse najvišji donos, kar je 16 iz enega recepta.

### Primarni in sekundarni materiali

Sedaj bi radi definirali razliko med materiali: tisti, ki jih lahko naredimo iz česa drugega in tisti, ki jih ne moremo (na primer drva, kamen ipd.). Za to bomo definirali razred, ki bo hranil podatke o predmetih, da jih lahko nato primerjamo:

In [46]:
class Item():
    def __init__(self, section: str, name: str) -> None:
        self.name = name
        self.section = section
        self.recipes = []
        self.material_for = set()
        self.is_primary = False
        self.is_final = False
        self.is_craftable = True
        self.primary_ingredients = []
        # For each primary material, there is a tuple in the form of (Ingredient, Amount)
        self.items_crafted = []
        # A list of all the items, that can be crafted from this (directly or indirectly)
        self.recipe_chains = []
        # Contains lists of items, ranging from this item to its primary materials
    
    def new_recipe(self, ings: str, ams: str, y: str):
        recipe = {}
        ings = ings.split(";")
        ams = ams.split(";")
        for i in range(len(ams)):
            recipe[ings[i]] = ams[i]
        recipe["yield"] = y
        self.recipes.append(recipe)
    
    def new_material(self, item: str):
        self.material_for.add(item)

Vsak objekt bo shranil podatke o tem predmetu iz CSV datoteke in potem se dodaten podatek: v receptih za katere predmete je uporabljen.

Pri iskanju primarnih materialov je še ena ovira; nekatere primarne materiale je mogoče "stisniti" v kocko tega materiala, to kocko pa je potem mogoče razdreti nazaj na prvotno količino materiala (Na primer železo: 9 palic železa lahko spremenimo v kocko železa, to pa lahko spremenimo nazaj v 9 železnih palic).

Takšnih materialov je več, in lahko bi to zaznali tako, da če je nek predmet material svojega materiala, ga potem označimo kot primarni material, a ne želimo, da bi bili kot primarna materiala označeni tako kocka kot palica železa, zato bom ročno sestavil seznam takšnih primarnih materialov:

In [47]:
primaries = ["Iron_Ingot", "Gold_Ingot", "Diamond", "Coal", "Lapis_Lazuli", "Raw_Iron", "Raw_Gold", "Netherite_Ingot"]

Sedaj shranimo vsak predmet v CSV tabeli v svoj predmet razreda Item in te shranimo v slovar, da lahko lažje dostopamo do njih:

In [48]:
items = {}

for i, row in csv_data.iterrows():
    #print(row)
    items[row["Name"]] = Item(row["Section"], row["Name"])
    items[row["Name"]].new_recipe(row["Ingredients"], row["Amounts"], row["Yield"])
    if row["Name"] in primaries:
        items[row["Name"]].is_primary = True

Sedaj, ko imamo vse predmete shranjene, lahko za vsak predmet preverimo, v katerem receptu je material za drug predmet:

In [50]:
materials = {}

for item in items.keys():
    for recipe in items[item].recipes:
        for material in recipe.keys():
            if material not in items.keys():
                # Če material v receptu še ni v slovarju predmetov, to pomeni,
                # da nima recepta, ki bi naredil ta predmet, torej je primarni material
                materials[material] = Item("Primary", material)
                materials[material].is_primary = True
                materials[material].is_craftable = False
            # Vsakemu materialu v vsakem receptu dodamo v seznam predmetov,
            # ki se jih da narediti iz tega materiala, predmet, ki je posledica tega recepta
                materials[material].new_material(item)
            else:
                items[material].new_material(item)
items = materials | items

Preverimo, da program pravilno zazna vse primarne materiale (če predmet nima recepta in je v receptu, potem je primarni material):

In [54]:
print("Predmeti, ki nimajo recepta in niso primarni:")
for item in items.keys():
    if len(items[item].recipes) == 0 and not items[item].is_primary:
        print(item, items[item].material_for)
print("Konec!")

Predmeti, ki nimajo recepta in niso primarni:
Konec!


Ker je presek med množico predmetov, ki niso primarni materiali, in množico predmetov brez recepta, prazna, naš program deluje pravilno.

Sedaj lahko zelo preprosto tudi definiramo končne predmete, to so tisti predmeti, ki niso material za noben drug predmet:

In [ ]:
for item in items.keys():
    if len(items[item].material_for) == 0:
        items[item].is_final = True

Tega ne bomo preverjali, saj je precej samoumevno.

## Veriga receptov

Sedaj, ko imamo te podatke na voljo, lahko tvorimo verige receptov. Zanima nas, Katere vse materiale moramo narediti, da izdelamo vsak predmet. Poleg tega bi bilo priročno vedeti, koliko katerih primarnih materialov potrebujemo za poljuben predmet, in za katere predmete moramo uporabiti posamezen primarni element.